In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

def science_plot(fontsize = 9):
    import scienceplots
    plt.style.use(['science','grid','notebook'])
    plt.rcParams.update({
        'font.size'       : fontsize,    # General font size
        'axes.titlesize'  : fontsize,    # Font size of the axes title
        'axes.labelsize'  : fontsize,    # Font size of the axes labels
        'xtick.labelsize' : fontsize,    # Font size of the x-axis tick labels
        'ytick.labelsize' : fontsize,    # Font size of the y-axis tick labels
        'legend.fontsize' : fontsize,    # Font size of the legend
        'figure.titlesize': fontsize,    # Font size of the figure title
        'legend.fancybox' : False,       # Disable the fancy box for legend
        'legend.edgecolor': 'k',         # Set legend border color to black
        'text.usetex'     : True,        # Use LaTeX for text rendering
        'font.family'     : 'serif'      # Set font family to serif
    })
science_plot()

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv
from matplotlib.colors import PowerNorm
from matplotlib.patches import Circle

def accurate_poisson_spot(diameter=2e-3, wavelength=632.8e-9, distance=1.0, 
                         screen_size=10e-3, resolution=1024):
    """
    Calculate the accurate Poisson spot diffraction pattern including obstacle visualization.
    """
    # Create coordinate system
    x = np.linspace(-screen_size/2, screen_size/2, resolution)
    y = np.linspace(-screen_size/2, screen_size/2, resolution)
    X, Y = np.meshgrid(x, y)
    r = np.sqrt(X**2 + Y**2)
    
    # Wave number and Fresnel number
    k = 2 * np.pi / wavelength
    N = diameter**2 / (4 * wavelength * distance)
    
    # Calculate diffraction pattern
    u = diameter * r / (wavelength * distance)
    intensity = np.zeros_like(r)
    mask = u != 0  # Avoid division by zero at center
    
    # Lommel functions approximation
    intensity[mask] = (2 * jv(1, np.pi * u[mask]/2) / (np.pi * u[mask]/2)
    intensity = np.abs(intensity)**2  # Convert to intensity
    
    # Poisson spot at center
    intensity[~mask] = 1.0
    
    # Create obstacle shadow region
    obstacle_radius_pixels = int((diameter/2) / (screen_size) * resolution)
    obstacle_mask = r < diameter/2
    intensity[obstacle_mask] = 0  # Perfectly dark shadow
    
    # Add slight penumbra effect at edges
    penumbra_width = 3  # pixels
    edge_mask = (r >= diameter/2) & (r < diameter/2 + penumbra_width*(screen_size/resolution))
    intensity[edge_mask] *= np.linspace(0, 1, penumbra_width)[np.newaxis, :]
    
    return intensity, x, obstacle_mask

def plot_poisson_system(intensity, coordinates, obstacle_mask, wavelength, diameter, distance):
    """Plot the complete system with obstacle and diffraction pattern."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Convert coordinates to millimeters
    mm_coords = coordinates * 1e3
    extent = [mm_coords.min(), mm_coords.max(), mm_coords.min(), mm_coords.max()]
    
    # Plot 1: Full system visualization
    sys_img = np.ones_like(intensity)
    sys_img[obstacle_mask] = 0.3  # Darker for obstacle
    
    # Create circular obstacle representation
    obstacle = plt.Circle((0, 0), diameter/2 * 1e3, color='black', fill=True)
    
    im1 = ax1.imshow(sys_img, cmap='gray', extent=extent)
    ax1.add_patch(obstacle)
    ax1.set_title('Obstacle and Light Source')
    ax1.set_xlabel('Position (mm)')
    ax1.set_ylabel('Position (mm)')
    ax1.text(0, diameter/2 * 1e3 + 0.5, f'{diameter*1e3:.1f} mm obstacle', 
             ha='center', va='bottom', color='white', fontsize=10)
    
    # Plot 2: Diffraction pattern
    im2 = ax2.imshow(intensity, cmap='hot', norm=PowerNorm(gamma=0.4), extent=extent)
    ax2.set_title('Diffraction Pattern with Poisson Spot')
    ax2.set_xlabel('Position (mm)')
    cbar = plt.colorbar(im2, ax=ax2)
    cbar.set_label('Normalized Intensity')
    
    # Highlight Poisson spot
    ax2.plot(0, 0, 'bo', markersize=8, fillstyle='none', label='Poisson Spot')
    ax2.legend(loc='upper right')
    
    plt.suptitle(f'Poisson Spot Simulation (λ = {wavelength*1e9:.1f} nm, Distance = {distance:.1f} m)')
    plt.tight_layout()
    plt.show()

def plot_cross_section(intensity, coordinates, diameter):
    """Plot intensity cross-section with obstacle region marked."""
    center_idx = intensity.shape[0] // 2
    cross_section = intensity[center_idx, :]
    
    plt.figure(figsize=(10, 5))
    
    # Plot intensity
    plt.plot(coordinates * 1e3, cross_section, 'r-', linewidth=2, label='Intensity')
    
    # Mark obstacle region
    obstacle_range = (-diameter/2 * 1e3, diameter/2 * 1e3)
    plt.axvspan(obstacle_range[0], obstacle_range[1], color='black', alpha=0.2, label='Obstacle')
    
    plt.title('Intensity Cross-Section Through Poisson Spot')
    plt.xlabel('Position (mm)')
    plt.ylabel('Normalized Intensity')
    plt.grid(True)
    plt.legend()
    
    # Highlight Poisson spot
    peak_idx = np.argmax(cross_section)
    plt.annotate('Poisson Spot', 
                 xy=(coordinates[peak_idx]*1e3, cross_section[peak_idx]), 
                 xytext=(coordinates[peak_idx]*1e3+1, cross_section[peak_idx]),
                 arrowprops=dict(facecolor='black', shrink=0.05),
                 fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Set matplotlib backend
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend for stability

# Simulation parameters
wavelength = 632.8e-9  # He-Ne laser wavelength
diameter = 2e-3  # 2 mm obstacle
distance = 1.0  # 1 meter to screen

try:
    # Run simulation
    intensity, coords, obstacle_mask = accurate_poisson_spot(
        diameter=diameter, 
        wavelength=wavelength, 
        distance=distance
    )

    # Plot results
    plot_poisson_system(intensity, coords, obstacle_mask, wavelength, diameter, distance)
    plot_cross_section(intensity, coords, diameter)
    
except Exception as e:
    print(f"Error: {str(e)}")
    print("Saving figures to files instead...")
    
    plt.figure()
    plt.imshow(intensity, cmap='hot', norm=PowerNorm(gamma=0.4))
    plt.colorbar()
    plt.savefig('poisson_spot_system.png')
    plt.close()
    
    plt.figure()
    center_idx = intensity.shape[0] // 2
    plt.plot(coords * 1e3, intensity[center_idx, :])
    plt.savefig('poisson_cross_section.png')
    plt.close()
    
    print("Figures saved to 'poisson_spot_system.png' and 'poisson_cross_section.png'")

SyntaxError: '(' was never closed (3082382670.py, line 28)